# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. HARSHA K
2. SANTOSH V BHAT
3. A N SRINIVASAN
4. YASH LODHA

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

**PEAS Environment for the Problem:**

***Performance Measure:***
> _Minimized communication lines with un-interrupted network._

***Environment:***
> _Shared and distributed servers placed at various locations in the city. Communication lines that connects all these servers._

***Actuators (O/P):***
> _Network agents selecting minimal communication lines to obtain un-interrupted network._

***Sensors (I/P):***
> _Sensors in this environment provide information about the current network topology at given instance, including the connections between servers and their statuses._

Design the agent as PSA Agent(Problem Solving Agent) 
Clear Initial data structures to define the graph and variable declarations is expected 
IMPORTANT: Write distinct code block as below

In [17]:
#Code Block : Set Initial State (Must handle dynamic inputs)

from enum import Enum, auto

class Server(Enum):
    A = 0
    B = auto()
    C = auto()
    D = auto()
    E = auto()
    F = auto()
    G = auto()
    H = auto()

# Graph is constructed statically as per given topology
graph = {
    Server.A: [(Server.B, 15), (Server.C, 10), (Server.D, 17), (Server.G, 5)],
    Server.B: [(Server.A, 15), (Server.D, 12)],
    Server.C: [(Server.A, 10), (Server.G, 7)],
    Server.D: [(Server.A, 17), (Server.B, 12), (Server.E, 2), (Server.F, 10), (Server.H, 4)],
    Server.E: [(Server.D, 2)],
    Server.F: [(Server.D, 10), (Server.H, 11)],
    Server.G: [(Server.A, 5), (Server.C, 7), (Server.H, 25)],
    Server.H: [(Server.D, 4), (Server.F, 11), (Server.G, 25)],
}

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0, h=0):
        self.state = state
        self.parent = parent # parent node
        self.action = action # action taken to reach this node
        self.path_cost = path_cost # path cost to reach this node g(n)

def create_dynamic_graph():
    dynamic_graph = {}


    import re

        #edge format
    edge_format = re.compile(r'([A-Z]) ([A-Z]) (\d+)')  
        
        #Input the edges
   #print("Enter the edges in the format <Node1 Node2 Cost> or <exit> to quit")
        
    while True:
        #Takes the input from the user
        edge_input = input("Enter the edges in the format <Node1 Node2 Cost> or <exit> to quit")
        #Breaks the loop if the input is exit
        if edge_input == 'exit':
            break
                
        match = edge_format.match(edge_input)   
        if match:
            #Assignes the 3 group(2 edges and path cost) values to node1, node2, cost
            node1, node2, cost = match.groups()
            #convert string to int    
            cost = int(cost)

            # Convert input nodes to Server enum members
            node1 = Server[node1]
            node2 = Server[node2]


            #If node1 is not present in graph add node1 in graph dictionary
            #Assign the value of this to empty list which represents neigbouring node and cost betweent them
            if node1 not in dynamic_graph:
                dynamic_graph[node1] = []
                #If node2 is not present in graph add node2 in graph dictionary
                #Assign the value of this to empty list which represents neigbouring node and cost betweent them
            if node2 not in dynamic_graph:
                dynamic_graph[node2] = []
            #By default node2 and cost is added as value to graph dictionary
                dynamic_graph[node1].append((node2, cost))
            #By default node1 and cost is added as value to graph dictionary
                dynamic_graph[node2].append((node1, cost))
            else:
                print("Invalid Entry. Please try again")
    return dynamic_graph

def select_graph_type():
    global graph
    graph_type = input("Enter 'static' for static graph or 'dynamic' for dynamic graph: ").strip().lower()
    if graph_type == 'static':
        pass  # Use the pre-defined static graph
    elif graph_type == 'dynamic':
        graph = create_dynamic_graph()
        
    else:
        print("Invalid input. Please enter 'static' or 'dynamic'.")
        select_graph_type()
    return graph_type, graph


# Get and validate the Node from user input
def get_input_node(message):
    # Valid inputs are A, B, C, D, E, F, G, H.
    server_input = input(message)
    # Convert the input string to the corresponding enum member
    try:
        server = Server[server_input]
        print("Server accepted:", {server.name})
    except KeyError:
        print("Error: ", {server_input}, " is not a valid server (A-H), try again...")
        server = get_input_node(message)
    return server

graph_type, graph = select_graph_type()
print(f"Selected graph type is {graph_type} and graph is {graph}")


# Initial/S0/Agent-in/start state
start_state = get_input_node("Enter the node where agent is present: ")
goal_state = get_input_node("Enter the node where agent should reach/goal node: ")

print("\nStart state: ", start_state)
print("Goal state: ", goal_state)

Selected graph type is dynamic and graph is {<Server.A: 0>: [(<Server.B: 1>, 12)], <Server.B: 1>: [(<Server.A: 0>, 12)]}
Error:  {'A '}  is not a valid server (A-H), try again...
Server accepted: {'B'}
Server accepted: {'B'}

Start state:  Server.B
Goal state:  Server.B


In [18]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

import numpy as np

# # Below transition cost matrix is derived from given topology/graph
# transition_cost = [
#     [0, 15, 10, 17, np.inf, np.inf, 5, np.inf],
#     [15, 0, np.inf, 12, np.inf, np.inf, np.inf, np.inf],
#     [10, np.inf, 0, np.inf, np.inf, np.inf, 7, np.inf],
#     [17, 12, np.inf, 0, 2, 10, np.inf, 4],
#     [np.inf, np.inf, np.inf, 2, 0, np.inf, np.inf, np.inf],
#     [np.inf, np.inf, np.inf, 10, np.inf, 0, np.inf, 11],
#     [5, np.inf, 7, np.inf, np.inf, np.inf, 0, 25],
#     [np.inf, np.inf, np.inf, 4, np.inf, 11, 25, 0]
# ]
# transition_cost_matrix = np.matrix(transition_cost)

# print(transition_cost_matrix)

# Derive transition cost matrix from given graph
def get_transition_cost_matrix(graph, servers):
    transition_cost = np.full((len(servers), len(servers)), np.inf)
    for server, connections in graph.items():
        for connection, cost in connections:
            transition_cost[server.value][connection.value] = cost
    for i in range(len(servers)):
        transition_cost[i][i] = 0
    return transition_cost

transition_cost_matrix = get_transition_cost_matrix(graph, list(Server))

print("Transition cost matrix:")
print(transition_cost_matrix)

Transition cost matrix:
[[ 0. 12. inf inf inf inf inf inf]
 [12.  0. inf inf inf inf inf inf]
 [inf inf  0. inf inf inf inf inf]
 [inf inf inf  0. inf inf inf inf]
 [inf inf inf inf  0. inf inf inf]
 [inf inf inf inf inf  0. inf inf]
 [inf inf inf inf inf inf  0. inf]
 [inf inf inf inf inf inf inf  0.]]


In [ ]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

from collections import deque

# def step_cost(state, action):
#     # Using graph
#     # Return the cost of taking the given action in the given state
#     for neighbor, cost in graph[state]:
#         if neighbor == action:
#             return cost
#     return 0

# def actions(state):
#     # Using graph
#     # Return a list of possible actions from the given state
#     actions = []
#     for neighbor, _ in graph[state]:
#         actions.append(neighbor)
#     print("Nodes generated from ", state, ":", actions)
#     return actions

# def result(state, action):
#     # Using graph
#     # Return the state that results from taking the given action in the given state
#     for neighbor, _ in graph[state]:
#         if neighbor == action:
#             return neighbor
#     return state

def step_cost(state, action):
    # Using transition matrix
    # Return the cost of taking the given action in the given state
    return transition_cost_matrix[state.value, action.value]

def actions(state):
    # Using transition matrix
    # Return a list of possible actions from the given state
    row = transition_cost_matrix[state.value]
    indices = np.where(row != np.inf)[0]
    actions = [Server(index) for index in indices]
    print("Nodes generated from", state, ":", actions)
    return actions

def result(state, action):
    # Using transition matrix
    # Return the state that results from taking the given action in the given state
    if transition_cost_matrix[state.value, action.value] != np.inf:
        return action
    return state

def transition_model(parent, action):
    # Return the state that results from taking the given action in the given state
    state = result(parent.state, action)
    path_cost = parent.path_cost + step_cost(parent.state, action)
    return Node(state, parent, action, path_cost)

def solution(node):
    # Return the sequence of actions to go from the root to given node.
    action_list = deque()
    path_cost = node.path_cost
    while node.parent is not None:
        action_list.appendleft(node.action.name)
        node = node.parent
    # Finally add the start state to the actions list
    action_list.appendleft(node.state.name)
    return {'path': list(action_list), 'path_cost': path_cost}

# Print the solution and path cost
def display_path(input):
    path = input['path']
    print("Path: " + " -> ".join(path))
    print("Path Cost: ", input['path_cost'])

In [ ]:
#Code block : Write function to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

def goal_test(state):
    res = state == goal_state
    print(f"Goal Test {state} : {'Pass' if res else 'Fail'}")
    return res

In [ ]:
# For heuristic design, consider all the possible paths between any arbitrary node n to the goal node.
# The average of the total transmission cost across all these paths is the heuristic value h(n)

import numpy as np
from collections import deque

def calculate_heuristic(node):
    # Calculate the heuristic value for the given node
    total_cost = 0
    num_paths = 0

    # Use Depth-First Search (DFS) to find all paths from the current node to the goal node
    stack = deque([(node.state, [node.state])])
    while stack:
        current_node, path = stack.pop()
        if current_node == goal_state:
            # Calculate the path cost and update the total cost
            path_cost = calculate_path_cost(path)
            total_cost += path_cost
            num_paths += 1
        else:
            # Explore the neighbors of the current node
            for neighbor, _ in graph[current_node]:
                if neighbor not in path:
                    stack.append((neighbor, path + [neighbor]))

    # Calculate the average cost across all paths
    if num_paths > 0:
        heuristic_value = total_cost / num_paths
    else:
        heuristic_value = 0

    return heuristic_value

def calculate_path_cost(path):
    # Calculate the total cost of a given path
    total_cost = 0
    for i in range(len(path) - 1):
        current_node = path[i]
        next_node = path[i + 1]
        total_cost += transition_cost_matrix[current_node.value, next_node.value]
    return total_cost

# Calculate heuristic values
h_values = {server: calculate_heuristic(Node(server)) for server in Server}
for h_value in h_values:
    print(f"Node: {h_value}, Heuristic: {h_values[h_value]}")

### 2.	Definition of Algorithm 1 (UNINFORMED SEARCH - BREADTH FIRST SEARCH)

In [ ]:
#Code Block : Function for algorithm 1 implementation

# TODO: Note 1, point 4 is pending

def breadth_first_search(node):
    # Check if the initial node is the goal
    if goal_test(node.state):
        return solution(node)
    
    # Initialize the frontier with the initial node
    frontier = deque([node])

    # Initialize the explored set to keep track of visited states (to avoid looping problem)
    explored = set()

    # Loop until the frontier is empty
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)

        # Iterate over all possible actions from the current state
        for action in actions(node.state):
            # Generate the child node by applying the action
            child = transition_model(node, action)

            if child.state not in explored and child not in frontier:
                # Check if the child node is the goal
                if goal_test(child.state):
                    return solution(child)
                
                # Add the child node to the frontier
                frontier.append(child)

    # If no solution is found, return an empty path
    return {'path': [], 'path_cost': np.inf}

### 3.	Definition of Algorithm 2 (INFORMED SEARCH - Recursive Best First Search)

In [ ]:
#Code Block : Function for algorithm 2 implementation

# f(n) = g(n) + h(n), initially assigned to 0's, transition_model and the algorithm updates as required.
f_values = {Server.A: 0, Server.B: 0, Server.C: 0, Server.D: 0, Server.E: 0, Server.F: 0, Server.G: 0, Server.H: 0}

# Do we need to remove explored set somewhere for truncation?
# Should the overriding of f_values be a problem in recursion?

def recursive_best_first_search(node, f_limit, explored):
    if goal_test(node.state):
        return {'status': True, 'solution': solution(node)}

    # Adding node to explored set to avoid looping problem
    explored.add(node.state)

    # Generate successors of the current node
    successors = []
    for action in actions(node.state):
        child = transition_model(node, action)
        # To avoid looping problem, only add child if it hasn't been explored
        if child.state not in explored:
            successors.append(child)

    # Printing the successors
    for s in successors:
        print(f"Successors of {node.state}: Node: {s.state}, Heuristic: {h_values[s.state]}, Path Cost: {s.path_cost}, f: {f_values[s.state]}")

    # If there are no successors, return failure with infinite cost
    if not successors:
        return {'status': False, 'solution': float('inf')}
    
    # Update f-values for all successors
    for s in successors:
        f_values[s.state] = max(s.path_cost + h_values[s.state], f_values[node.state])

    while True:
        # Successor with the lowest f-value
        best = min(successors, key=lambda x: f_values[x.state])

        # If the best f-value exceeds the f_limit, return failure with the best f-value
        if f_values[best.state] > f_limit:
            return {'status': False, 'solution': f_values[best.state]}

        # Successor with the second-lowest f-value
        alternative = min(successors, key=lambda x: f_values[x.state] if x != best else float('inf'))

        print("Best path node: ", best.state, "Alternative/Second-best path node: ", alternative.state)

        # Call recursively with the best successor and new f_limit
        output = recursive_best_first_search(best, min(f_limit, f_values[alternative.state]), explored)

        # Base condition: If the recursive call returns a solution, send it upwards
        if output['status'] != False:
            return output

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question. 

In [ ]:
#Code Block : Function & call to get inputs (start/end state)

### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Invoke the breadth first search algorithm
path = breadth_first_search(Node(start_state, None, None, 0))
display_path(path)

# TODO: Note 1, point 2: "Compare to interpret the results in terms of the algorithm working, performance & shortest path"
# TODO: Note 1, point 4: "Include code in your implementation to calculate the space complexity and time complexity and print the same."

In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Initialize the explored set to keep track of visited states (to avoid looping problem)
explored = set()
# Invoke the recursive best first search algorithm
output = recursive_best_first_search(Node(start_state, None, None, 0), float('inf'), explored)
if output['status']:
    display_path(output['solution'])
else:
    print("No path")

# TODO: Note 1, Descr  : "heuristic function implementation is pending"
# TODO: Note 1, point 2: "Compare to interpret the results in terms of the algorithm working, performance & shortest path"
# TODO: Note 1, point 3: "For each incremental depth limit, corresponding node generating sequence should be legibly printed in the result" (If same actions() is used, then it is already implemented)
# TODO: Note 1, point 4: "Include code in your implementation to calculate the space complexity and time complexity and print the same."


### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 1 

#TODO: Testing

# V = len(Server)  # Number of vertices
# E = np.count_nonzero(transition_cost_matrix != np.inf)  # Number of edges

# time_complexity = f"O({V} + {E})"
# space_complexity = f"O({V})"

# print(f"Time Complexity: {time_complexity}")
# print(f"Space Complexity: {space_complexity}")


import math

def calculate_depth(n, b):
    if b == 1:
        # If branching factor is 1, the tree is a linear chain
        return n - 1
    
    return math.log(n * (b - 1) + 1, b) - 1


V = len(Server)  # Number of vertices
E = np.count_nonzero(transition_cost_matrix != np.inf)  # Number of edges
non_leaf_nodes = sum(1 for node_edges in graph.values() if node_edges)
b = E // non_leaf_nodes  # Approximate branching factor (edges per vertex)
    
n = V  # Number of nodes
d = calculate_depth(n, b)

time_complexity = f"O({b}^{d})"
space_complexity = f"O({b}^{d})"

print(f"Time Complexity: {time_complexity}")
print(f"Space Complexity: {space_complexity}")
print(f"Depth of the tree: {d:.2f}")

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2

### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________